## Bridging combat messages spreadsheet to JSON 

In [19]:
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
from rdflib.namespace import XSD, RDF, OWL, RDFS
from rdflib import Graph, URIRef, Namespace, BNode, Literal
import json
import re
import shortuuid 

In [20]:
in_file = 'Bridging_Combat_Messages_Breakdown_v2.xlsx'
out_file = 'Bridging_Combat_Messages_Breakdown_v2.ttl'

In [21]:
# namespaces
cm = "http://purl.org/artiamas/cm/"
CM = Namespace(cm)

In [22]:
file = load_workbook(filename = in_file)
sheet = file.active

In [23]:
# read the spreadsheet data into a JSON-like python structure
all_data = []

last_column = len(list(sheet.columns))
last_row = len(list(sheet.rows))

for row in range(2, last_row + 1):
    data = {}
    data['sequence'] = row-1
    for column in range(2, last_column + 1):
        col = get_column_letter(column)
        property = sheet[col + str(1)].value
        value = sheet[col + str(row)].value
        if row > 1:
            data[property] = value
    all_data.append(data)

In [51]:
# custom BBode function adds a prefix to a short uuid sequence
def bnode(prefix = ''):
    if prefix:
        return BNode(prefix + '_' + shortuuid.uuid()[:10])
    else:
        return BNode(shortuuid.uuid()[:8])

In [52]:
# dictionary mapping a spreadsheet column name to a RDF property URI
str2property = {'Time':CM.time, 'Agent':CM.agent, 'sequence':CM.sequence, 
                'Type':CM.messageType, 'Level':CM.level, 'Grid':CM.grid,
                'Agent':CM.agent, 'Target':CM.target, 'Alert Messages':CM.alertMessage, 'id':CM.id}

In [53]:
# function mapping prop's value to a RDF property URI
def str2object (msg, prop, value):
    """ convert a string into a object or literal """
    if type(value) in [int, float]:
        return Literal(value)
    elif prop == 'Agent':
        return parse_agent(value)
    elif prop == 'Target':
        if value and " Task " in value:
            # sometimes the Target field has the task
            g.add((task, RDFS.label, Literal(value)))
            return task
        elif not msg['Agent']:
            return parse_agent(value)
        else:
            return parse_target(value, msg=msg)
    else:
        return Literal(value)

**Given a string (e.g., "B CO / 1 - 22") representing a military unit we've not seen before, we parse the string to infer the unit's type (e.g., CM.Company) and its superior units (e.g., Batallion 1; Regiment 22) nd their types, adding information to the RDF graph. The function returns the sgent's URI**

In [54]:
def parse_target(text, field='target', infer_types=False, msg=None):
    return parse_agent(text, field='target', infer_types=infer_types)

def parse_agent(text, field='agent', infer_types=False, msg=None):
    """returns """
    global instances
    # noramlize agent references
    if not text:
        return CM.NONE
    name = text.strip()
    if not name:
        return CM.NONE
    if '/' not in name:
        # sometimes there's just the local name
        name = name + ' / 1 / 22 IN'
    name = name.replace('-','/').replace(' ','_')
    instance = URIRef(cm + name)

    if name in instances:
        # We've seen this before, so just return the instance
        return instances[name]

    id = bnode('UNIT')
    instances[name] = id
    g.add((id, RDF.type, CM.MilitaryUnit))
    g.add((id, RDFS.label, Literal(name)))
    if field == 'agent':
        g.add((id, CM.force, CM.BLUE))
    elif field == 'target' and msg and msg['Agent']:
        g.add((id, CM.force, CM.RED))
    
    if not infer_types:
        return id
    
    unit_type = infer_unit_type(name)
    
    platoonid = None
    
    if re.search('^\d_\d_CO', name):
        platoonid = cm + text
        text = text[2:]
      
    units = [x.strip('_') for x in text.split('/')]
    
    unitid = cm + unit
    if 'ENG' in unit:
        unit_type = CM.EngineeringCompany
    elif 'SCT_PLT' in unit:
        unit_type = CM.ScoutPlatoon
    elif 'MORTAR_PLT' in unit:
        unit_type = CM.MortarPlatoon
    elif 'CO' in unit:
        unit_type = CM.Company
    else:
        print(f"Assuming {unit} is of type cm:Company")
        unit_type = CM.Company
        
    battalionid = cm + rest
    battalion_type = CM.Battalion

    regimentid = cm + regiment
    if 'IN' in regiment:
        regiment_type = CM.InfantryRegiment
    else:
        regiment_type = CM.Regiment
        
    # add instance data
    if platoonid:
        g.add((platoonid, RDF.type, CM.Platoon))
        g.add((platoonid, CM.unitOf, unitid))
    
    g.add((unitid, RDF.type, unit_type))
    g.add((unitid, CM.unitOf, battalionid))
    
    g.add((battalionid, RDF.type, battalion_type ))
    g.add((battalionid, CM.unitOf, regimentid))
    
    g.add((regimentid, RDF.type, regiment_type))
    
    return instance

In [62]:
def add_action(msg, msg_id):
    """ add an action to the message"""
    act = bnode('ACT')
    alert = msg["Alert Messages"].lower()
    agent = list(g.objects(msg_id, CM.agent))[0]
    target = list(g.objects(msg_id, CM.target))[0]
    g.add((msg_id, CM.action, act))
    g.add((act, CM.time, list(g.objects(msg_id, CM.time))[0] ))
    g.add((act, CM.sequence, list(g.objects(msg_id, CM.sequence))[0] ))
    g.add((act, CM.alertMessage, Literal(alert)))
    if 'resupply' in alert:
        g.add((act, RDF.type, CM.Resupply))
        g.add((act, CM.recipient, target))
        if "(ammo)" in alert:
            g.add((act, CM.object, CM.AMMO))
        elif "(fuel)" in alert:
            g.add((act, CM.object, CM.FUEL))
    elif 'earned sa' in alert:
        g.add((act, RDF.type, CM.EarnedSA))
        g.add((act, CM.subject, agent))
        g.add((act, CM.object, target))
    elif 'moving to fight' in alert:
        g.add((act, RDF.type, CM.Move))
        g.add((act, CM.subject, agent))
        g.add((act, CM.toward, target))
        g.add((act, CM.reason, URIRef("attack")))
    elif re.match('attacking.*against', alert):
        g.add((act, RDF.type, CM.Attack))
        g.add((act, CM.subject, agent))
        g.add((act, CM.object, target))
    elif 'fighting' in alert:
        g.add((act, RDF.type, CM.StartFight))
        g.add((act, CM.subject, agent))
        g.add((act, CM.toward, target))
    elif re.match('attacking.*ended', alert):
        g.add((act, RDF.type, CM.EndFight))
        g.add((act, CM.subject, agent))
        g.add((act, CM.toward, target))
    elif re.match('receiving.*fire$', alert):
        g.add((act, RDF.type, CM.Attack))
        g.add((act, CM.object, agent))
    elif re.match('receiving.*fire ended', alert):
        g.add((act, RDF.type, CM.EndAttack))
        g.add((act, CM.object, agent))
    elif "not going after opfor" in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.subject, agent))
        g.add((act, CM.Status, CM.UNABLE ))
        g.add((act, CM.reason, Literal(alert)))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.Engage))
        g.add((act1, CM.subject, agent))
        g.add((act1, CM.object, target))
    elif "paused at crossing control point" in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.Status, CM.PAUSE ))
        if "crossing is not traversable" in alert:
            g.add((act, CM.reason, Literal("Crossing is not traversable")))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.MOVE))
        g.add((act1, CM.subject, agent))
        g.add((act1, CM.object, target))
    elif 'task' in alert or 'Task' in msg['Alert Messages']:
        # g.add((act, RDF.type, CM.EndAttack))
        g.add((act, RDF.type, CM.TaskReport))
        g.add((act, CM.subject, agent))
        # g.add((act, CM.object, agent))
    elif 'waiting' in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.subject, agent))
        g.add((act, CM.Status, CM.WAIT ))
        if "can't attack without reinforcements" in alert:
            g.add((act, CM.reason, Literal("need reinforcements")))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.Attack))
        g.add((act1, CM.subject, agent))
        g.add((act1, CM.object, target))
    elif 'resume' in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.subject, agent))
        g.add((act, CM.Status, CM.RESUME))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.Move))
        g.add((act1, CM.subject, agent))
        g.add((act1, CM.object, target))
    elif 'planned battle removed' in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.subject, agent))
        g.add((act, CM.Status, CM.END))
        if 'no real targets' in alert:
            g.add((act, CM.reason, Literal("no targets")))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.Attack))
        g.add((act1, CM.subject, agent))
        g.add((act1, CM.object, target))
    elif 'firing has stopped'in alert:
        g.add((act, RDF.type, CM.Report))
        g.add((act, CM.subject, agent))
        g.add((act, CM.Status, CM.END ))
        act1 = bnode('ACT')
        g.add((act, CM.action, act1 ))
        g.add((act1, RDF.type, CM.Attack))
        g.add((act1, CM.object, agent))
    else:
        print("Unrecognized alert:", alert)

In [63]:
# Create the inital RDF graph 
g = Graph()
CM = Namespace('http://purl.org/artiamas/cm/')
g.bind("cm", CM, override=True)
g.bind("owl", OWL, override=True)
g.bind("rdf", RDF, override=True)
instances = {} # dict of names to bnodes
task = bnode('TASK')
g.add((task, RDF.type, CM.CrossingTask))

<Graph identifier=N842cecb10c7046739708e778e14724e9 (<class 'rdflib.graph.Graph'>)>

In [64]:
messages = []
actions = []
for msg in all_data:
    subj = bnode('MSG')
    messages.append(subj)
    g.add((subj, RDF.type, CM.CombatMessage))
    g.add((subj, CM.task, task))
    for prop, obj in msg.items():
        if prop not in str2property:
            print(f"Unrecognized property {prop}")
            continue
        obj = str2object(msg, prop, obj)
        prop = str2property[prop]
        g.add((subj, prop, obj))
    add_action(msg, subj)
g.add((task, CM.firstMessage, messages[0]))
g.add((task, CM.lastMessage, messages[-1]))
for i in range(len(messages)):
     g.add((messages[i], CM.nextMessage, messages[1+1]))

In [65]:
g.serialize(destination=out_file)

<Graph identifier=N842cecb10c7046739708e778e14724e9 (<class 'rdflib.graph.Graph'>)>

In [66]:
print(g.serialize())

@prefix cm: <http://purl.org/artiamas/cm/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a cm:CombatMessage ;
    cm:action [ a cm:Resupply ;
            cm:alertMessage "log resupply [thin air] [logpac] (ammo)" ;
            cm:object cm:AMMO ;
            cm:recipient _:UNIT_9CxDW6mYe3 ;
            cm:sequence 8 ;
            cm:time "H+03:00" ] ;
    cm:agent cm:NONE ;
    cm:alertMessage "Log resupply [thin air] [LOGPAC] (ammo)" ;
    cm:grid "38T PK 68497 35744" ;
    cm:level "ROUTINE" ;
    cm:messageType "LOG" ;
    cm:nextMessage _:MSG_JU4HRqpPwJ ;
    cm:sequence 8 ;
    cm:target _:UNIT_9CxDW6mYe3 ;
    cm:task _:TASK_m3hVAfVVkG ;
    cm:time "H+03:00" .

[] a cm:CombatMessage ;
    cm:action [ a cm:Report ;
            cm:Status cm:UNABLE ;
            cm:action [ a cm:Engage ;
                    cm:object _:UNIT_DmKBJNL2VA ;
                    cm:subject _:UNIT_WyQEYq9Lkm ] ;
            cm:alertMessage 

In [140]:
for name in instances.keys():
    print(name)

SCT_PLT_/_1_/_22_IN
1_1_CO_(TRK)_/_1_BN_/_241_INF_BDE_(TM)
C_CO_/_1_/_22_IN
50_ENG_CO_(MRBC)_/_1_/_22_IN
A_CO_/_1_/_22_IN
D_CO_/_1_/_22_IN
MORTAR_PLT_/_1_/_22_IN
B_CO_/_1_/_22_IN
3_1_CO_(TRK)_/_1_BN_/_241_INF_BDE_(TM)
2_1_CO_(TRK)_/_1_BN_/_241_INF_BDE_(TM)
